In [15]:
# # Required Packages for the Project RUN ONLY ONCE
# %pip install matplotlib
# %pip install nilearn
# %pip install openpyxl
# %pip install Path
# %pip install seaborn
# %pip install nltools
# %pip install scipy
# %pip install scikit-image
# %pip install nibabel
# %pip install bctpy

In [16]:
# import copy
# from skimage import filters
# import json
# import nibabel as nib
# from nilearn.plotting import plot_img

import pandas as pd
from nilearn import image
from nilearn.image import load_img, index_img
import numpy as np
import scipy.io
import networkx as nx
import matplotlib as mt
import scipy as sc
import seaborn as sns
import bct
from copy import deepcopy

In [17]:
class SubjectData:
    
    def __init__(self, datasetPath, componentsPath):
        self.componentFileName = "adni_aa__sub01_component_ica_s1_.nii"
        self.timeCourseFileName = "adni_aa__sub01_timecourses_ica_s1_.nii"
        self.FNCmatFile = "adni_aa__postprocess_results.mat"
        self.component_key = "fnc_corrs_all"
        self.graphMetricGlobalMeasues = [
                # Global Measures
                "Global efficiency", 
                "Characteristic path length", 
                "Clustering coefficient",
            ]
        self.graphMetricComponentMeasures = [
                # Component Measures
                "Degree",
                "Closeness centrality",
                "Participation coefficient"
            ]
        self.subjectsData = self.readFileofCSV(datasetPath)
        self.componentData = self.readFileofCSV(componentsPath)

        self.modifySubjectsPath()

        self.domainList = self.componentData["icn_domain"]
        self.indexList = self.componentData["icn_index"]

    def modifySubjectsPath(self):
        for i in range(2404):
            self.subjectsData.at[i,"fc_dir"] = self.subjectsData.iloc[i]["fc_dir"].replace("FC","GIGICA")

    def readFileofCSV(self,path):
        fileData = pd.read_csv(path)
        return fileData
    
    def getDatasetPaths(self,subjectName):
        ans = list()
        for i in range(2404):
            if self.subjectsData.at[i,"ResearchGroup"] == subjectName:
                ans.append(self.subjectsData.at[i,"fc_dir"])
        return ans

    def calculatePearsonCrossCorrelations(self, listA, listB):
        xmean = np.mean(listA)
        xval = listA - xmean
        xsqu = np.sqrt(np.sum(np.square(xval)))

        ymean = np.mean(listB)
        yval = listB - ymean
        ysqu = np.sqrt(np.sum(np.square(yval)))

        num = 0
        for k in range(len(yval)):
            num += (xval[k]*yval[k])

        den = (xsqu * ysqu)
        ans = num/den
        return ans

In [18]:
class VoxelCounts(SubjectData):
    " \
        datasetPath : 4D images path \
        componentsPath: indexes path \
        subjectList: List of subjects to calculate ex: AD, CN, ....  \
    "
    def __init__(self,datasetPath, componentsPath, subjectList):
        super().__init__(datasetPath,componentsPath)
        self.subjectsToCalculate = subjectList

        self.voxelCountMap = dict()
        self.prepareVoxelCountMap()

    def prepareVoxelCountMap(self):
        " \
            iterate over the list of Subjects like AD, CN, ....     \
        "
        for subjectName in self.subjectsToCalculate:
            self.voxelCountMap[subjectName]=dict()
            self.voxelCountMap[subjectName]["paths"] = self.getDatasetPaths(subjectName)
            self.voxelCountMap[subjectName]["indexes"] = dict()
            for i in self.indexList:
                self.voxelCountMap[subjectName]["indexes"][i]=list()

    def calculateVoxelCount(self):
        for subjectName in self.subjectsToCalculate:
            for path in self.voxelCountMap[subjectName]["paths"]:
                spacialMapName = path + self.componentFileName
                spacialMap = load_img(spacialMapName)
                for index in self.voxelCountMap[subjectName]["indexes"]:
                    actualIndex=index-1
                    componentImg = index_img(spacialMap, actualIndex)
                    componentImgData = componentImg.get_fdata()
                    component_threshold = 3*np.std(componentImgData)
                    component_voxelCount = np.count_nonzero(componentImgData > component_threshold)
                    self.voxelCountMap[subjectName]["indexes"][index].append(component_voxelCount)

In [19]:

AD_Threshold = VoxelCounts('ADNI_demos.txt', 'NM_icns_info.csv', [ "CN", "MCI",  "AD" ] )
AD_Threshold.calculateVoxelCount()

In [20]:
class GraphMetrics(SubjectData):

    " \
        datasetPath : 4D images path \
        componentsPath: indexes path \
        subjectList: List of subjects to calculate ex: AD, CN, ....  \
    "

    def __init__(self,datasetPath, componentsPath, subjectList):
        super().__init__(datasetPath,componentsPath)
        self.subjectsToCalculate = subjectList

        self.graphMetricMap = dict()
        self.prepareGraphMetricMap()

    def prepareGraphMetricMap(self):
        for subjectName in self.subjectsToCalculate:
            self.graphMetricMap[subjectName]=dict()
            self.graphMetricMap[subjectName]["paths"] = self.getDatasetPaths(subjectName)
            for graphMetric in self.graphMetricGlobalMeasues:
                self.graphMetricMap[subjectName][graphMetric]=list()

            for graphMetric in self.graphMetricComponentMeasures:
                self.graphMetricMap[subjectName][graphMetric] = dict()
                for ind in self.indexList:
                    self.graphMetricMap[subjectName][graphMetric][ind]=list()

    def loadMatFile(self,filePath):
        componentDict = scipy.io.loadmat(filePath+self.FNCmatFile)
        return componentDict[self.component_key]

    def prepareFNCMatrix(self,componentData):
        selected_component = np.zeros((53,53), dtype=np.float64).reshape(53,53)
        for i in range(53):
            for j in range(i+1, 53):
                # if componentData[self.indexList[i]-1][self.indexList[j]-1] >=0 : 
                selected_component[i][j]=componentData[self.indexList[i]-1][self.indexList[j]-1]
        selected_component += selected_component.T

        # Finding correlation matrix
        corrs = pd.DataFrame(selected_component)
        correlation_matrix = corrs.corr()
        correlation_numpy_array = correlation_matrix.to_numpy(dtype=np.float64)

        # Finding Thresholded correlation matrix
        row,column = correlation_numpy_array.shape
        for i in range(row):
            for j in range(column):
                if i!=j and correlation_numpy_array[i][j]<0:
                    correlation_numpy_array[i][j]=0

        input_matrix = nx.from_numpy_array(correlation_numpy_array)
        return input_matrix

    def calculateGraphMetrics(self):
        for subjectName in self.subjectsToCalculate:
            for path in self.graphMetricMap[subjectName]["paths"]:
                fncMatrix = self.prepareFNCMatrix(self.loadMatFile(path))

                # for global measures:
                for graphMetric in self.graphMetricGlobalMeasues:
                    if graphMetric == "Global efficiency":
                        globalEfficieny = nx.global_efficiency(fncMatrix)
                        self.graphMetricMap[subjectName][graphMetric].append(globalEfficieny)

                    elif graphMetric == "Characteristic path length":
                        characteristicPathLength = nx.average_shortest_path_length(fncMatrix, weight='weight')
                        self.graphMetricMap[subjectName][graphMetric].append(characteristicPathLength)

                    elif graphMetric == "Clustering coefficient":
                        clusterCofficient = nx.clustering(fncMatrix, weight='weight')
                        clusterCofficient_network = 0
                        for i in clusterCofficient:
                            clusterCofficient_network += clusterCofficient[i]
                        clusterCofficient_network /= 53
                        self.graphMetricMap[subjectName][graphMetric].append(clusterCofficient_network)

                    else:
                        print("Not Found: ", graphMetric)
                        assert(False)

                # for component measures
                for graphMetric in self.graphMetricComponentMeasures:
                    if graphMetric == "Degree":
                        all_nodes_degree = nx.degree(fncMatrix, weight= 'weight')
                        for ind in range(len(self.indexList)):
                            self.graphMetricMap[subjectName][graphMetric][self.indexList[ind]].append(all_nodes_degree[ind])

                    elif graphMetric == "Closeness centrality":
                        all_nodes_cc = nx.closeness_centrality(fncMatrix, distance='weight', wf_improved=False)
                        for ind in range(len(self.indexList)):
                            self.graphMetricMap[subjectName][graphMetric][self.indexList[ind]].append(all_nodes_cc[ind])

                    elif graphMetric == "Participation coefficient":
                        fncMatrix_numpy = nx.to_numpy_array(fncMatrix)
                        modularity = nx.algorithms.community.greedy_modularity_communities(fncMatrix)
                        sam=[]
                        for i in range(len(modularity)):
                            for j in list(modularity[i]):
                                sam.append(j)
                        sam = np.array(sam)
                        all_nodes_pc = bct.centrality.participation_coef(fncMatrix_numpy, ci=sam, degree='undirected')
                        for ind in range(len(self.indexList)):
                            self.graphMetricMap[subjectName][graphMetric][self.indexList[ind]].append(all_nodes_pc[ind])

                    else:
                        print("Not Found: ", graphMetric)
                        assert(False)

In [21]:

AD_GraphMetric = GraphMetrics('ADNI_demos.txt', 'NM_icns_info.csv', [ "CN", "MCI", "AD" ])
AD_GraphMetric.calculateGraphMetrics()

In [22]:
# For Global specific measures.

GlobalGraphMetrics_Names = AD_GraphMetric.graphMetricGlobalMeasues
ComponentGraphMetric_Names = AD_GraphMetric.graphMetricComponentMeasures
ans_global = list()
ans_component = list()

for subjectName in AD_GraphMetric.subjectsToCalculate:

    table_global = pd.DataFrame()
    table_component = pd.DataFrame()
    for i in range(len(AD_Threshold.indexList)):
        table_global[AD_Threshold.domainList[i]+'('+str(AD_Threshold.indexList[i])+')'] = list()
        table_component[AD_Threshold.domainList[i]+'('+str(AD_Threshold.indexList[i])+')'] = list()

    for graphMetricName in GlobalGraphMetrics_Names:
        indexList = AD_Threshold.indexList
        corr_list_global = list()

        xlist_global = deepcopy(AD_GraphMetric.graphMetricMap[subjectName][graphMetricName])
        for j in range(len(indexList)):

            ylist_global = deepcopy(AD_Threshold.voxelCountMap[subjectName]["indexes"][AD_Threshold.indexList[j]])
            corr_list_global.append( AD_Threshold.calculatePearsonCrossCorrelations(xlist_global, ylist_global) )

        table_global.loc[len(table_global.index)] = corr_list_global


    for i in ComponentGraphMetric_Names:
        indexList = AD_Threshold.indexList
        corr_list_component = list()

        for j in range(len(indexList)):
 
            xlist_component = deepcopy(AD_GraphMetric.graphMetricMap[subjectName][i][indexList[j]])
            ylist_component = deepcopy(AD_Threshold.voxelCountMap[subjectName]["indexes"][AD_Threshold.indexList[j]])
            corr_list_component.append(AD_Threshold.calculatePearsonCrossCorrelations(xlist_component, ylist_component))

        table_component.loc[len(table_component.index)] = corr_list_component

    table_global.index = AD_GraphMetric.graphMetricGlobalMeasues
    table_component.index = AD_GraphMetric.graphMetricComponentMeasures
    ans_global.append(table_global)
    ans_component.append(table_component)

In [43]:
temp_global = deepcopy(ans_global)
temp_component = deepcopy(ans_component)

In [ ]:
symvalue = 0.5
fileNames = ['Controls', 'Mild_Cognetive_Impairment', 'Alzheimers']

for i in range(len(fileNames)):
        # temp_global[i].to_csv(fileNames[i]+'_global.csv')

        s1 = temp_global[i].to_numpy()
        fig, ax = mt.pyplot.subplots(figsize=(54,15))
        columnsName = temp_global[i].columns.values
        rowsName = temp_global[i].index.values
        sd = sns.heatmap(
                        s1, cmap='coolwarm', square=True, ax=ax, linewidth='0.5', center=0, vmin= -1*symvalue, vmax=symvalue,
                        xticklabels=columnsName, yticklabels=rowsName,  annot=True
                )
        ax.set_title(fileNames[i]+str(symvalue)+'_global.png')
        sd.get_figure().savefig(fileNames[i]+str(symvalue)+'_global.png')

for i in range(len(fileNames)):
        # temp_component[i].to_csv(fileNames[i]+'_component.csv')

        s1 = temp_component[i].to_numpy()
        fig, ax = mt.pyplot.subplots(figsize=(54,15))
        columnsName = temp_component[i].columns.values
        rowsName = temp_component[i].index.values
        sd = sns.heatmap(
                        s1, cmap='coolwarm', square=True, ax=ax, linewidth='0.5', center=0, vmin= -1*symvalue, vmax=symvalue,
                        xticklabels=columnsName, yticklabels=rowsName,  annot=True
                )
        ax.set_title(fileNames[i]+str(symvalue)+'_component.png')
        sd.get_figure().savefig(fileNames[i]+str(symvalue)+'_component.png')

In [45]:
def findDifferenceCrossCorrelation(name1, name2, name3, tab1, tab2):
    diff = pd.DataFrame().reindex_like(tab1).dropna()
    columns = tab1.columns.values

    rows = tab1.index.values

    for i in rows:
        for j in columns:
            diff.at[i,j] = tab1.at[i,j] - tab2.at[i,j]

    st1 = diff.to_numpy()

    fig, ax = mt.pyplot.subplots(figsize=(54,15))

    sd = sns.heatmap (
                    st1, 
                    vmin= -1*symvalue, 
                    vmax= symvalue,
                    cmap='coolwarm', 
                    square=True,
                    ax=ax,
                    linewidth='0.5',
                    center=0, 
                    cbar=True,
                    xticklabels=columnsName, 
                    yticklabels=rowsName,  
                    annot=True
                )
    ax.set_title(name1+'-'+name2+'-'+name3+str(symvalue)+'.png')
    sd.get_figure().savefig(name1+'-'+name2+'-'+name3+str(symvalue)+'.png')


In [ ]:
sizeN = len(fileNames)

for i in range(sizeN):
    for j in range(i+1,sizeN):
        findDifferenceCrossCorrelation(fileNames[i], fileNames[j], 'global', temp_global[i], temp_global[j])
        findDifferenceCrossCorrelation(fileNames[i], fileNames[j], 'component', temp_component[i], temp_component[j])

In [48]:
import os
import random
from PIL import Image, ImageOps

def concat_images(image_paths, size, shape=None):
    # Open images and resize them
    width, height = size
    images = map(Image.open, image_paths)
    images = [ImageOps.fit(image, size, Image.ANTIALIAS) 
              for image in images]
    
    # Create canvas for the final image with total size
    shape = shape if shape else (1, len(images))
    image_size = (width * shape[1], height * shape[0])
    image = Image.new('RGB', image_size)
    
    # Paste images into final image
    for row in range(shape[0]):
        for col in range(shape[1]):
            offset = width * col, height * row
            idx = row * shape[1] + col
            image.paste(images[idx], offset)
    
    return image

def createImage(keyword):
    # Get list of image paths
    folder = '.'
    for key in keyword:
        image_array = [ i+str(symvalue)+'_'+key+'.png' for i in fileNames ]
        for i in range(sizeN):
            for j in range(i+1,sizeN):
                image_array.append(fileNames[i]+'-'+fileNames[j]+'-'+key+str(symvalue)+'.png')
        image = concat_images(image_array, (3888,1080), (3,2))
        image.save('image_'+key+'.png', 'PNG')

createImage(['global', 'component'])

In [ ]:
"""
1 4d subject
    - 53 selected Components
        - each component voxel Count - index i

AD- 213
69  - [ 1_Sub_69Ind_voxelCount, 2_Sub_69Ind_VoxelCount,  ]
53  - [ ]

1 4D subject:
    - preprocess.mat file 
        - FNC matrix
            - Graph metric 
                - Glo
                - Charac
                - Coefficint

AD- 213
Global Efficieny - [ 1_Sub_mat_FNC_Global_Value, 2_Sub_mat_FNC_Global, ..... ]
Characterstic Path lenght - [ 1_sub, ........ ]


Correlation:
Global Efficiency Vs 69 [] - Correlation 0.123
Global Efficiency Vs 53 [] - Correlation -0.324
                                            0.372


"""